In [8]:
import cv2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
# ビデオデータの読み込み
filename = "C:\\Users\\flow\\Desktop\\bubbledate\\tetrahedronGray.mp4"
base_path= "C:\\Users\\flow\\Desktop\\bubble_rasing_result\\"
# 動画ファイルのベース名を取得
video_base_name = os.path.splitext(os.path.basename(filename))[0]

# 動画ごとのディレクトリを作成
video_directory = os.path.join(base_path, video_base_name)
os.makedirs(video_directory, exist_ok=True)  # 既に存在している場合は作成しない
# 新しいフォルダのベース名を指定
folder_names = ["d_image", "d_contours", "d_result", "d_threshold", "d_graph"]

# 新しいフォルダを作成
for folder_name in folder_names:
    new_folder_path = os.path.join(video_directory, folder_name)
    os.makedirs(new_folder_path, exist_ok=True)

# 動画の読み込み
cap = cv2.VideoCapture(filename=filename)
#動画ファイルの情報を取得する
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
hight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print("総フレーム数: ", total_frame)
print("fps: ", fps)
print("動画の長さ: ", total_frame/fps, "[sec]")

frame_count = 0  # フレームのカウントを初期化
prev_cx = None  # 前のフレームの重心 x 座標
prev_cy = None  # 前のフレームの重心 y 座標
# グラフ用のデータ格納リスト
area_data = []
perimeter_data = []
moment_data = []
# 重心距離のデータを格納するリスト
centroid_distance_data = []
speed_data = []  # 速度情報を格納するリスト、最初の値を0に初期化
# 重心の座標を格納するリスト
centroid_x_list = []
centroid_y_list = []
centroid_data = []
# 正三角形の底辺と高さのデータを格納するリスト
base_lengths = []
heights = []

# 正三角錐の体積データを格納するリスト
cone_volumes = []

# 得られた球の半径を格納するリスト
sphere_radiuses = []

# 正三角錐の体積を計算する関数
def calculate_cone_volume(base, height):
    return (1/3) * base**2 * height

# 開始フレームを設定
start_frame = 30  # 例として30フレーム目から開始

# 開始フレームまで読み込む
for _ in range(start_frame):
    ret, _ = cap.read()
    if not ret:
        print("Error: Failed to read frames.")
        sys.exit(1)

while cap.isOpened() and frame_count < 90:  # 90枚分の処理を行う
    ret, frame = cap.read()
    if ret:   
        num = frame_count
        path = os.path.join(video_directory, "d_image\\")
        contours_path = os.path.join(video_directory, "d_contours\\")
        result_path = os.path.join(video_directory, "d_result\\")
        threshold_path = os.path.join(video_directory, "d_threshold\\")
        
        image_path = path + "picture{:0=3}.bmp".format(num)
        cv2.imwrite(image_path, frame)

        color_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if color_image is None:
            print(f"Error: Failed to load image from {image_path}")
            sys.exit(1)

        gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
        blurred_image = cv2.bilateralFilter(gray_image, 7, 15, 15 )
        ret, binary = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        thresholds_path = threshold_path + "threshold_picture{:0=3}.bmp".format(num)
        cv2.imwrite(thresholds_path, binary)

        raw_contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        raw_img_with_contours = frame.copy()
        cv2.drawContours(raw_img_with_contours, raw_contours, -1, (0, 255, 0), 2)
        raw_contours_image_path = os.path.join(contours_path, "raw_contours_picture{:0=3}.bmp".format(num))
        cv2.imwrite(raw_contours_image_path, raw_img_with_contours)

        contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        #print(contours)
        conditions = {
            "min_contour_area": 10000,
            "max_contour_area": 20000,
        }

        filtered_contours = []
        area_data = []
        perimeter_data = []
        moment_data = []

        for idx, contour in enumerate(contours):
            contour_area = cv2.contourArea(contour)

            if (
                ("min_contour_area" not in conditions or contour_area >= conditions["min_contour_area"]) and
                ("max_contour_area" not in conditions or contour_area <= conditions["max_contour_area"])
            ):
                filtered_contours.append(contour)
                area_data.append(contour_area)
                perimeter_data.append(cv2.arcLength(contour, True))
                M = cv2.moments(contour)
                filtered_cx = int(M['m10'] / M['m00']) if M["m00"] != 0 else 0
                filtered_cy = int(M['m01'] / M['m00']) if M["m00"] != 0 else 0
                moment_data.append((filtered_cx, filtered_cy))

        if len(filtered_contours) > 0:
            result_image = color_image.copy()
        
            for contour, moment in zip(filtered_contours, moment_data):
                x_coordinates = [point[0][0] for point in contour]
                y_coordinates = [point[0][1] for point in contour]
                
                min_x = min(x_coordinates)
                max_x = max(x_coordinates)
                min_y = min(y_coordinates)
                max_y = max(y_coordinates)
                base_length = max_x - min_x
                height = max_y - min_y

                base_lengths.append(base_length)
                heights.append(height)

                cone_volume = calculate_cone_volume(base_length, height)
                cone_volumes.append(cone_volume)

                # 体積に相当する球の半径を算出
                sphere_radius = ((3 * cone_volume) / (4 * math.pi))**(1/3)
                sphere_radiuses.append(sphere_radius)

                M = cv2.moments(contour)
                if M["m00"] != 0:
                    filtered_cx, filtered_cy = moment
                    area = cv2.contourArea(contour)
                    # 重心の座標をリストに追加
                    centroid_x_list.append(filtered_cx)
                    centroid_y_list.append(filtered_cy)
                    
                    # 面積を描画
                    #cv2.putText(result_image, f"Area: {area}", (filtered_cx, filtered_cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                    
                    # 重心に点を描画
                    cv2.circle(result_image, (filtered_cx, filtered_cy), 5, (0, 0, 255), -1)
                    # 輪郭を描画
                    cv2.drawContours(result_image, [contour], -1, (0, 255, 0), 2)
            
            result_image_path = os.path.join(result_path, "result_picture{:0=3}.bmp".format(num))
            cv2.imwrite(result_image_path, result_image)
        
        # フレームごとに速度情報を格納
        if prev_cx is not None and prev_cy is not None:
            dx = filtered_cx - prev_cx
            dy = filtered_cy - prev_cy
            centroid_distance = math.sqrt(dx ** 2 + dy ** 2)
            centroid_distance_data.append(centroid_distance)
            dt = 1 / fps  # フレーム間の時間差（秒）
            speed_px_per_s = math.sqrt(dx ** 2 + dy ** 2) / dt
            speed_data.append(speed_px_per_s)
            
            # 重心のデータを追加
            centroid_data.append((filtered_cx, filtered_cy))
        
        # 前のフレームの重心を更新
        prev_cx = filtered_cx
        prev_cy = filtered_cy

        frame_count += 1
    else:
        break


cap.release()  # カメラリソースの解放
cv2.destroyAllWindows()  # ウィンドウの破棄

# グラフを作成
plt.figure(figsize=(10, 6))  # グラフのサイズを設定

# 重心距離のデータをプロット
plt.plot(centroid_distance_data, label='Centroid Distance', color='blue')

# 速度のデータをプロット
plt.plot(speed_data, label='Speed', color='red')

# グラフにタイトルと軸ラベルを追加
plt.title('Centroid Distance and Speed Over Frames')
plt.xlabel('Frame Number')
plt.ylabel('Value')

# 凡例を表示
plt.legend()

# グラフを保存
graph_filename = os.path.join(video_directory, "d_graph", "centroid_speed_graph.png")
plt.savefig(graph_filename)

# グラフを表示
plt.show()



総フレーム数:  449.0
fps:  29.97002997002997
動画の長さ:  14.981633333333333 [sec]


IndexError: invalid index to scalar variable.